In [1]:
import requests
import xmltojson
import pandas as pd
import io
import csv
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import json 

In [2]:
n = 10
page_URL = 'https://www.consejo-fisioterapia.org/vu_colegiados/pag_5/descargar.html'
main_URL = 'https://www.consejo-fisioterapia.org/vu_colegiados.html'


In [3]:

page = requests.get(main_URL)
page.headers['Content-Type']

'text/html; charset=UTF-8'

In [4]:
# parse to beatiful
soup = BeautifulSoup(page.content, "html.parser")
# Find where the number of pages are storted


In [5]:

results = soup.find(id="contenido")
# Search for the specific class that stores the number of pages 



In [6]:
job_elements = results.find_all("ul", class_="pagination")
# Iterate through elements to get the links (even if it's just one element)
job_elements

[<ul class="pagination">
 <!--
 					<li>
 						<a href="#" title="Anterior" aria-label="Anterior">
 							<span aria-hidden="true">&laquo;</span>
 						</a>
 					</li>
 					-->
 <li></li>
 <li></li>
 <li class="active"><span class="active">1</span></li>
 <li> <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_2.html">2</a> <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_3.html">3</a> <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_4.html">4</a></li>
 <li> <a>...</a> <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_1961.html">1961</a></li>
 <!--
 					<li>
 						<a href="#" title="Siguiente" aria-label="Siguiente">
 							<span aria-hidden="true">&raquo;</span>
 						</a>
 					</li>
 					-->
 </ul>]

In [7]:

for element in job_elements:
    # -- snip --
    links = element.find_all("a")
links

[<a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_2.html">2</a>,
 <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_3.html">3</a>,
 <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_4.html">4</a>,
 <a>...</a>,
 <a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_1961.html">1961</a>]

In [8]:
links[-1]

<a href="https://www.consejo-fisioterapia.org/vu_colegiados/pag_1961.html">1961</a>

In [9]:
# get the last page reference 
last_page = int(links[-1].text)
last_page

1961

In [9]:
main_URL = 'https://www.consejo-fisioterapia.org/vu_colegiados.html'
# Define the URL that has the information of one specific page. 
page_URL = 'https://www.consejologopedas.com/censo_de_colegiados/pag_1/descargar.html'
# Make a request to the page 
response = requests.get(main_URL)
# Check if request is satifactory 
if response:
    print('Request succesfull, proceeding to extract the number of pages...')
else:
    print(f'Response not sucesfull. Error: {response}')
# Parse to beatifulsoup so we can operate
soup = BeautifulSoup(response.content, "html.parser")
# Find in which section the pages are defined 
results = soup.find(id="contenido")
# Search for the specific class that stores the number of pages 
job_elements = results.find_all("ul", class_="pagination")
        
# Iterate through elements to get the links (even if it's just one element)
for element in job_elements:
    # -- snip --
    links = element.find_all("a")
# get the last page reference 
last_page = int(links[-1].text)
print(f'Last page found: {last_page}')


Request succesfull, proceeding to extract the number of pages...
Last page found: 1961


In [41]:
# Iterate through all the number of pages and download the script
for i in tqdm(range(1,5)):
    # Actualize the data 
    url_tmp = re.sub(r'\d', str(i), page_URL)
    # request the csv file
    urlData = requests.get(url_tmp, timeout=10)
    # Check if request it's Ok
    if urlData:
        # check if it's the first time and create the original dataframe
        if i == 1:
            # Transform the data into a dataframe and create main dataframe
            df = pd.read_csv(io.StringIO(urlData.content.decode('ISO-8859-1')), delimiter=';')
        else: 
            # Transform the data dataframe and create a temporary df
            df_tmp = pd.read_csv(io.StringIO(urlData.content.decode('ISO-8859-1')), delimiter=';')
            #  concatenate the dataframe
            df = pd.concat(
                            [df, df_tmp],
                            axis=0,
                            join="outer",
                            ignore_index=True,
                            keys=None,
                            levels=None,
                            names=None,
                            verify_integrity=False,
                            copy=True,
                        )
    else:
        print(f'An Error: {urlData} ocurred while requesting the information on tha mage URL: \n{page_URL}')

100%|██████████| 4/4 [00:06<00:00,  1.69s/it]


In [42]:
df.drop(columns=df.columns[-1], inplace=True)

In [44]:
df1 = df.copy()

In [58]:
df = df1.copy()
df

,COLEGIO,NUMERO COLEGIADO,NOMBRE,EJERCE
0,Colegio Oficial de Logopedas de Andalucía,1,María del Carmen Martín Garrido,Ejerce
1,Colegio Profesional de Logopedas de Galicia,1,Ana Isabel. Vázquez Lojo,Ejerce
2,Colegio de Logopedas de las Islas Baleares,1,Miquela Maria Sastre Vidal,Ejerce
3,Colegio Profesional de Logopedas de la Comunid...,1,MARÍA CRISTINA MUNICIO VENTURA,Ejerce
4,Colegio Profesional de Logopedas de Extremadura,1,Arturo Domínguez Giralt,Ejerce
...,...,...,...,...
115,Colegio Profesional de Logopedas de la Comunid...,1031,MARTA MERCHÁN GUINALDO,Ejerce
116,Colegio Oficial de Logopedas de Andalucía,1032,Rosario Pérez Muñoz,Ejerce
117,Colegio Profesional de Logopedas de la Comunid...,1032,NURIA MORENO NUÑEZ,Ejerce
118,Colegio Oficial de Logopedas de Andalucía,1033,Paloma Rocío Miranda Ponce,No ejerce


In [60]:
label = df.EJERCE.unique()[0]
label_2 = df.EJERCE.unique()[1]
label, label_2


('Ejerce', 'No ejerce')

In [61]:
df.loc[df["EJERCE"] == label, "EJERCE"] = 1
df.loc[df["EJERCE"] == label_2, "EJERCE"] = 0
df

,COLEGIO,NUMERO COLEGIADO,NOMBRE,EJERCE
0,Colegio Oficial de Logopedas de Andalucía,1,María del Carmen Martín Garrido,1
1,Colegio Profesional de Logopedas de Galicia,1,Ana Isabel. Vázquez Lojo,1
2,Colegio de Logopedas de las Islas Baleares,1,Miquela Maria Sastre Vidal,1
3,Colegio Profesional de Logopedas de la Comunid...,1,MARÍA CRISTINA MUNICIO VENTURA,1
4,Colegio Profesional de Logopedas de Extremadura,1,Arturo Domínguez Giralt,1
...,...,...,...,...
115,Colegio Profesional de Logopedas de la Comunid...,1031,MARTA MERCHÁN GUINALDO,1
116,Colegio Oficial de Logopedas de Andalucía,1032,Rosario Pérez Muñoz,1
117,Colegio Profesional de Logopedas de la Comunid...,1032,NURIA MORENO NUÑEZ,1
118,Colegio Oficial de Logopedas de Andalucía,1033,Paloma Rocío Miranda Ponce,0


In [62]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   COLEGIO           120 non-null    object
 1   NUMERO COLEGIADO  120 non-null    int64 
 2   NOMBRE            120 non-null    object
 3   EJERCE            120 non-null    object
dtypes: int64(1), object(3)
memory usage: 30.1 KB


In [63]:
df1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   COLEGIO           120 non-null    object
 1   NUMERO COLEGIADO  120 non-null    int64 
 2   NOMBRE            120 non-null    object
 3   EJERCE            120 non-null    object
dtypes: int64(1), object(3)
memory usage: 33.4 KB


In [67]:
df[df.columns[0]].unique()

array(['Colegio Oficial de Logopedas de Andalucía',
       'Colegio Profesional de Logopedas de Galicia',
       'Colegio de Logopedas de las Islas Baleares',
       'Colegio Profesional de Logopedas de la Comunidad de Madrid',
       'Colegio Profesional de Logopedas de Extremadura',
       'Col·legi de Logopedes de Catalunya'], dtype=object)